# Final Project NLP Smart Scientific Content Auto-completion System


a text auto-complete system that supports scientists in writing articles in:
- completing some typed words/letters.
- Identifying words/phrases that are very unlikely to occur in some text contexts and suggesting proper replacements.


Karmel Salah & Fatimah Najwan
15/4/2022

# We will use Colab for a Free GPU so if you are using Colab frist connect to your drive to get the dataset


In [1]:
# # Load the Drive helper and mount
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# #unzip the dataset folder
# !unzip ./drive/MyDrive/dataset.zip 

# Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import os, glob
import math
import matplotlib.pyplot as plt
import seaborn as sns
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec


[nltk_data] Downloading package punkt to /home/karmel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/karmel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/karmel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/karmel/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-05-24 08:16:23.518207: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such fil

In [4]:
# get the names of each folders
files_names = os.listdir('./dataset')
len(files_names)

1064

# Convert to DataFrame

In [5]:
df = pd.DataFrame([], columns = ["id", "articles"])
print(f"data frame is {df}")

data frame is Empty DataFrame
Columns: [id, articles]
Index: []


In [6]:
def convert_to_df(df, files_names):
        id = []
        articles= []
        # Define files place and read each sub file for cleaning
        for index, file in enumerate(files_names):
                # read each article files
                path = os.getcwd()
                file_path = '{}/dataset/{}'.format(path,file)
                with open(file_path,'r', encoding='utf-16',  errors="ignore") as f:
                    texts = f.read() 
                    articles.append(texts) 
                    id.append(index)
                    f.close()
        print("the ids", id)
        print("the articles", articles)
        df['id'] = id
        df["articles"] = articles
        return df


In [7]:
df = convert_to_df(df, files_names)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
df["articles"]

0       Statistical Machine Translation\n\nDraft of Ch...
1       Found in Translation:\n\nLearning Robust Joint...
2       Available online at www.sciencedirect.com\n\nS...
3       J OURNAL OF I NFORMATION S CIENCE AND E NGINEE...
4       The University of Sheﬃeld\n\nT. E. Dunning\n\n...
                              ...                        
1059    Multi-label Hate Speech and Abusive Language D...
1060    hULMonA (      ): The Universal Language Model...
1061    Mazajak: An Online Arabic Sentiment Analyser\n...
1062    2017 2nd International Conferences on Informat...
1063    INVESTIGATIVE RADIOLOGY Volume 28, Number 6, 4...
Name: articles, Length: 1064, dtype: object

In [9]:
# get only 50 articles for our experiment
limited_df = pd.DataFrame(df['articles'].iloc[0:50])

In [10]:
# count the number of words for each doc
limited_df['word_count'] = limited_df['articles'].apply(lambda x: len(str(x).split()))

 # Text pre-processing:

In [11]:
import gensim, spacy, logging, warnings
from gensim.utils import simple_preprocess
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [12]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower()
    # Remove leading and trailing spaces
    text= text.strip()  
    text= re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text= re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    # get ride of one letter word!
    text = re.sub(r'\b\w\b','',text) 
    
      
    return text

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/karmel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

In [15]:
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [16]:
import enchant

In [17]:
#Remove non English words
dictionary = enchant.Dict("en_US")
   
def remove_non_english(text):
        en_words= []
        for word in text:
            if dictionary.check(word):
                en_words.append(word)
        text = ' '.join(str(e) for e in en_words)
        return text

In [18]:
# remove non English words
limited_df['english_text'] =limited_df['articles'].apply(lambda text: remove_non_english(text.split()))

In [19]:
#limited_df['english_text'][0]

In [20]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

In [21]:
# clean each article
limited_df['clean_text'] = limited_df['english_text'].apply(lambda x: finalpreprocess(x))


In [22]:
data_words = limited_df.clean_text.values.tolist()

In [23]:
print(data_words[0:1])

['statistical machine translation draft chapter neural machine translation center speech language processing department computer science john hopkins university st public draft august nd public draft september content neural machine translation short history introduction neural network linear model multiple layer non linearity inference back propagation training computation graph neural network computation graph gradient computation deep learn framework neural language model fee forward neural language model word embed inference training recurrent neural language model long short term memory model gate recurrent unit deep model neural translation model encoder decoder approach add alignment model train beam search content ensemble decode large vocabulary use monolingual data deep model guide alignment training model coverage adaptation add linguistic annotation multiple language pair alternate architecture neural network neural network attention self attention current challenge domain 

## Build the Bigram, Trigram Models and Lemmatize

In [24]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [25]:
!python3 -m spacy download en

2022-05-24 08:17:14.270046: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-24 08:17:14.270075: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: Crypt

In [26]:
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

In [27]:
data_ready = process_words(data_words) 

In [28]:
print(data_ready[0:1])

[['statistical', 'machine', 'translation', 'draft', 'chapter', 'translation', 'center', 'speech', 'language', 'processing', 'department', 'computer', 'public', 'draft', 'public', 'draft', 'neural', 'machine', 'translation', 'short', 'history', 'introduction', 'neural', 'network', 'linear', 'layer', 'non', 'linearity', 'inference', 'back', 'propagation', 'training', 'computation', 'graph', 'neural', 'network', 'computation', 'graph', 'gradient', 'computation', 'deep', 'learn', 'framework', 'neural', 'language', 'model', 'forward', 'neural', 'language', 'model', 'word', 'embe', 'inference', 'training', 'recurrent', 'neural', 'language', 'model', 'long', 'short', 'term', 'memory', 'model', 'gate', 'recurrent', 'unit', 'deep', 'model', 'neural', 'translation', 'model', 'encoder', 'decoder', 'approach', 'add', 'alignment', 'model', 'train', 'beam', 'search', 'content', 'ensemble', 'decode', 'large', 'vocabulary', 'monolingual', 'datum', 'deep', 'model', 'guide', 'alignment', 'training', 'mo

In [29]:
model = Word2Vec(data_ready, min_count=1)

In [30]:
# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

In [31]:
#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
        
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    


In [32]:
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))
modelw = MeanEmbeddingVectorizer(w2v)

# converting texts to numerical data using Word2Vec
vectors_w2v = modelw.transform(data_ready)

In [33]:
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models.coherencemodel import CoherenceModel
#vis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [34]:
# lets create our Dictionary
id2word = corpora.Dictionary(data_ready)

# Create our Corpus Term Doc Frequency
corpus = []
for text in data_ready:
    new = id2word.doc2bow(text)
    corpus.append(new)
    
    
# print first word from my corpus
print ("Word from Corpus", corpus[0][0:20])
# lets print the word and see the result
word = id2word[[0][:1][0]]
print ("Word from id2word", word)

Word from Corpus [(0, 1), (1, 1), (2, 2), (3, 14), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 2), (10, 1), (11, 2), (12, 9), (13, 1), (14, 1), (15, 6), (16, 2), (17, 3), (18, 32), (19, 2)]
Word from id2word abandon


In [35]:
# compute the ladmodel with the best parameter after the tunning
# alpha string
# for whole articles use 32 but for 50 articles I will use only 10
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics= 10,
                                           random_state = 100,
                                           update_every = 1,
                                           chunksize = 100,
                                           passes=10,
                                           alpha='symmetric',
                                           per_word_topics=True,
                                           eta = 0.7)

In [36]:
# Compute Coherence Score
coherencemodel = CoherenceModel(model=lda_model, texts=data_ready, dictionary=id2word, coherence='c_v')
coherence= coherencemodel.get_coherence()
print('\nCoherence Score: ', coherence)


Coherence Score:  0.4124980839751295


In [37]:
# lets see the Topics 
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.025*"measure" + 0.019*"class" + 0.014*"predict" + 0.014*"true" + 0.013*"set" + 0.012*"base" + 0.012*"pair" + 0.009*"hierarchical" + 0.009*"figure" + 0.008*"system"


Topic: 1 
Words: 0.019*"domain" + 0.016*"learn" + 0.011*"modality" + 0.011*"target" + 0.010*"representation" + 0.010*"source" + 0.010*"feature" + 0.009*"datum" + 0.008*"set" + 0.008*"adaptation"


Topic: 2 
Words: 0.014*"naive" + 0.013*"probability" + 0.012*"class" + 0.009*"baye" + 0.008*"text" + 0.006*"feature" + 0.006*"word" + 0.006*"conditional" + 0.005*"sample" + 0.005*"document"


Topic: 3 
Words: 0.014*"test" + 0.013*"model" + 0.009*"method" + 0.009*"system" + 0.009*"word" + 0.009*"likelihood" + 0.008*"document" + 0.008*"sequence" + 0.007*"information" + 0.007*"language"


Topic: 4 
Words: 0.033*"word" + 0.028*"model" + 0.013*"language" + 0.012*"vector" + 0.010*"train" + 0.009*"learn" + 0.008*"set" + 0.007*"datum" + 0.007*"representation" + 0.007*"table"


Topic: 5 
Words: 0.012*"walk" + 0.010*"gra

## Tunning the Parameters of LDA Model

In [38]:
# Find the optimal number of Topics for LDA

# def compute_coherence_values(dictionary, corpus,texts, limit, start,step,alpha, beta):
#     coherence_values = []
#     model_list = []
#     for num_topics in range(start, limit, step):
#         model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                            id2word=id2word,
#                                            num_topics=num_topics, 
#                                            random_state=100,
#                                            update_every=1,
#                                            chunksize=100,
#                                            passes=10,
#                                            alpha=alpha,
#                                            per_word_topics=True,
#                                            eta = beta)
#         model_list.append(model)
#         coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values

In [39]:
# Can take a long time to run.

#alpha_list = ['symmetric',0.3,0.5,0.7]
#beta_list = ['auto',0.3,0.5,0.7]


#for alpha in alpha_list:
        #for beta in beta_list:
            #model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=tok, start=2, limit=40, step=6,alpha=alpha,beta =beta)
            #print(f"alpha : {alpha} ; beta : {beta} ; Score : {coherence_values}")
            


In [40]:
# # Show graph
# limit=40; start=2; step=6;
# x = range(start, limit, step)
# plt.plot(x, coherence_values)
# plt.xlabel("Num Topics")
# plt.ylabel("Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

### End of the Tunning

# Understanding the LDA topic Model through DataFrame

# Finding the dominant topic in each sentence

In [41]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
            # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
                if j == 0:  # => dominant topic
                    wp = ldamodel.show_topic(topic_num)
                    topic_keywords = ", ".join([word for word, prop in wp])
                    sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                else:
                    break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,6.0,0.9999,"model, neural, network, word, translation, inp...","[statistical, machine, translation, draft, cha..."
1,1,1.0,0.6962,"domain, learn, modality, target, representatio...","[find, learn, robust, joint, representation, c..."
2,2,1.0,0.9993,"domain, learn, modality, target, representatio...","[available, online, system, engineering, route..."
3,3,9.0,0.9996,"term, weight, text, stem, scheme, document, ca...","[inverse, category, frequency, base, supervise..."
4,4,3.0,0.9999,"test, model, method, system, word, likelihood,...","[find, structure, genome, symbolic, sequence, ..."
5,5,9.0,0.8483,"term, weight, text, stem, scheme, document, ca...","[stem, ultra, stem, improve, automatic, text, ..."
6,6,7.0,0.9864,"sequence, part, word, mathematical, equation, ...","[detect, phrase, university, apply, informatio..."
7,7,4.0,0.6916,"word, model, language, vector, train, learn, s...","[fast, accurate, sentiment, classification, en..."
8,8,0.0,0.9998,"measure, class, predict, true, set, base, pair...","[evaluation, measure, hierarchical, view, nove..."
9,9,9.0,0.5489,"term, weight, text, stem, scheme, document, ca...","[generate, sequence, recurrent, neural, networ..."


In [42]:
len(df_dominant_topic)

50

In [43]:
# lets sort the dataframe according to the Dominant Topic
df_soreted = df_dominant_topic.sort_values(by='Dominant_Topic', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
df_soreted

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
8,8,0.0,0.9998,"measure, class, predict, true, set, base, pair...","[evaluation, measure, hierarchical, view, nove..."
1,1,1.0,0.6962,"domain, learn, modality, target, representatio...","[find, learn, robust, joint, representation, c..."
2,2,1.0,0.9993,"domain, learn, modality, target, representatio...","[available, online, system, engineering, route..."
37,37,1.0,0.9689,"domain, learn, modality, target, representatio...","[learn, research, publish, domain, adversarial..."
22,22,2.0,0.8527,"naive, probability, class, baye, text, feature...","[naive, bayes, text, introduction, theory, int..."
4,4,3.0,0.9999,"test, model, method, system, word, likelihood,...","[find, structure, genome, symbolic, sequence, ..."
35,35,3.0,0.7510,"test, model, method, system, word, likelihood,...","[relation, extraction, framework, wa, informat..."
48,48,4.0,0.7700,"word, model, language, vector, train, learn, s...","[language, model, engineering, apply, abstract..."
25,25,4.0,0.9994,"word, model, language, vector, train, learn, s...","[caption, visual, concept, abstract, paper, pr..."
26,26,4.0,0.9970,"word, model, language, vector, train, learn, s...","[effective, word, order, text, categorization,..."


In [44]:
# lets Get the topics numbers and articles
topics = df_soreted[['Dominant_Topic','Text']].values.tolist() 

In [45]:
topics_numbers = [item[0] for item in topics]

In [46]:
# lets put all articles grouped together by its topic number
grouped_df = df_soreted.groupby('Dominant_Topic').agg(list)

In [47]:
grouped_df.head()

,Document_No,Topic_Perc_Contrib,Keywords,Text
Dominant_Topic,,,,
0.0,[8],[0.9998000264167786],"[measure, class, predict, true, set, base, pai...","[[evaluation, measure, hierarchical, view, nov..."
1.0,"[1, 2, 37]","[0.6962000131607056, 0.9993000030517578, 0.968...","[domain, learn, modality, target, representati...","[[find, learn, robust, joint, representation, ..."
2.0,[22],[0.8526999950408936],"[naive, probability, class, baye, text, featur...","[[naive, bayes, text, introduction, theory, in..."
3.0,"[4, 35]","[0.9998999834060669, 0.7509999871253967]","[test, model, method, system, word, likelihood...","[[find, structure, genome, symbolic, sequence,..."
4.0,"[48, 25, 26, 27, 28, 29, 24, 33, 23, 39, 40, 4...","[0.7699999809265137, 0.9994000196456909, 0.996...","[word, model, language, vector, train, learn, ...","[[language, model, engineering, apply, abstrac..."


In [48]:
# lets change the Topic index name so we dont be confuse by the name
index = grouped_df.index
grouped_df['Topic_num'] = index

In [49]:
grouped_df['Topic_num']

Dominant_Topic
0.0    0.0
1.0    1.0
2.0    2.0
3.0    3.0
4.0    4.0
5.0    5.0
6.0    6.0
7.0    7.0
8.0    8.0
9.0    9.0
Name: Topic_num, dtype: float64

In [50]:
# get the result of the grouping of each Text with its topic number on different datagram
df_char = grouped_df[['Topic_num','Keywords', 'Text']]
df_char.head()

,Topic_num,Keywords,Text
Dominant_Topic,,,
0.0,0.0,"[measure, class, predict, true, set, base, pai...","[[evaluation, measure, hierarchical, view, nov..."
1.0,1.0,"[domain, learn, modality, target, representati...","[[find, learn, robust, joint, representation, ..."
2.0,2.0,"[naive, probability, class, baye, text, featur...","[[naive, bayes, text, introduction, theory, in..."
3.0,3.0,"[test, model, method, system, word, likelihood...","[[find, structure, genome, symbolic, sequence,..."
4.0,4.0,"[word, model, language, vector, train, learn, ...","[[language, model, engineering, apply, abstrac..."


In [51]:
# change the index name of the new dataframe as well so we wont get confused 
df_char.index.names = ['index']

In [52]:
df_char

,Topic_num,Keywords,Text
index,,,
0.0,0.0,"[measure, class, predict, true, set, base, pai...","[[evaluation, measure, hierarchical, view, nov..."
1.0,1.0,"[domain, learn, modality, target, representati...","[[find, learn, robust, joint, representation, ..."
2.0,2.0,"[naive, probability, class, baye, text, featur...","[[naive, bayes, text, introduction, theory, in..."
3.0,3.0,"[test, model, method, system, word, likelihood...","[[find, structure, genome, symbolic, sequence,..."
4.0,4.0,"[word, model, language, vector, train, learn, ...","[[language, model, engineering, apply, abstrac..."
5.0,5.0,"[walk, graph, random, vertex, representation, ...","[[online, learn, social, representation, compu..."
6.0,6.0,"[model, neural, network, word, translation, in...","[[collaborative, deep, learn, system, science,..."
7.0,7.0,"[sequence, part, word, mathematical, equation,...","[[detect, phrase, university, apply, informati..."
8.0,8.0,"[solver, factorization, fast, machine, python,...","[[learn, research, submit, revise, publish, fa..."


### Word Counts of Topic Keywords

In [53]:
# get the topics from the model and count the words of keywords
from collections import Counter

topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in data_ready for w in w_list]
counter = Counter(data_flat)

out = []
embedding_wieghts = []
for i, topic in topics:
    for word, weight in topic:
        embedding_wieghts.append(weight)
        out.append([word, i , weight, counter[word]])

In [54]:
len(topics)

10

In [55]:
embedding_wieghts = np.reshape(embedding_wieghts , (-1, 10))

In [56]:
embedding_wieghts

array([[0.02520594, 0.01918529, 0.01364142, 0.01352893, 0.01335979,
        0.01205861, 0.01171972, 0.00923755, 0.00901146, 0.00831845],
       [0.01928682, 0.01579707, 0.01060551, 0.01052721, 0.00998539,
        0.00989439, 0.00977355, 0.00931035, 0.00827993, 0.00763779],
       [0.01399363, 0.01258384, 0.01202483, 0.00929958, 0.00775381,
        0.00606169, 0.00583469, 0.00572149, 0.00546143, 0.00531534],
       [0.01444087, 0.01288268, 0.00943054, 0.00928264, 0.00889764,
        0.00868222, 0.00840844, 0.0078552 , 0.00742039, 0.00712041],
       [0.03332662, 0.02771569, 0.0131271 , 0.01162938, 0.01014967,
        0.00881109, 0.00846772, 0.00708055, 0.00702598, 0.00692899],
       [0.0118991 , 0.01003485, 0.00822909, 0.00807226, 0.00663862,
        0.00561219, 0.00517596, 0.00478393, 0.00445851, 0.00412425],
       [0.02363105, 0.01873809, 0.01751926, 0.01697313, 0.0144425 ,
        0.01134231, 0.01053981, 0.00964806, 0.00911196, 0.00904496],
       [0.01369436, 0.01286334, 0.0122499

In [57]:
# 10 Topics wieghts for each 10 keywords 
# get the max wieght of each topic 
topics_embedding_wieghts = []
for topic_w in embedding_wieghts:
    max = np.amax(topic_w)
    topics_embedding_wieghts.append(max)
    print(max)

0.025205942
0.019286824
0.013993629
0.014440872
0.03332662
0.011899096
0.023631053
0.013694364
0.0036966112
0.01875337


In [58]:
# All Keywords for all Topics
# ['Word', 'Topic id', 'wieght', 'counter']
out

[['measure', 0, 0.025205942, 421],
 ['class', 0, 0.019185288, 428],
 ['predict', 0, 0.013641417, 340],
 ['true', 0, 0.013528929, 176],
 ['set', 0, 0.01335979, 1063],
 ['base', 0, 0.012058615, 803],
 ['pair', 0, 0.011719716, 328],
 ['hierarchical', 0, 0.009237546, 154],
 ['figure', 0, 0.009011456, 590],
 ['system', 0, 0.008318447, 976],
 ['domain', 1, 0.019286824, 450],
 ['learn', 1, 0.015797071, 1271],
 ['modality', 1, 0.010605507, 139],
 ['target', 1, 0.010527206, 417],
 ['representation', 1, 0.009985393, 739],
 ['source', 1, 0.009894388, 491],
 ['feature', 1, 0.0097735515, 634],
 ['datum', 1, 0.009310347, 986],
 ['set', 1, 0.008279927, 1063],
 ['adaptation', 1, 0.0076377937, 140],
 ['naive', 2, 0.013993629, 94],
 ['probability', 2, 0.01258384, 472],
 ['class', 2, 0.012024833, 428],
 ['baye', 2, 0.009299585, 60],
 ['text', 2, 0.0077538146, 702],
 ['feature', 2, 0.0060616853, 634],
 ['word', 2, 0.0058346926, 2908],
 ['conditional', 2, 0.005721486, 118],
 ['sample', 2, 0.0054614334, 352

In [59]:
# save it to data frame
df_keywords = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])     

In [60]:
# Plot Word Count and Weights of Topic Keywords
# import seaborn as sns
# import matplotlib.colors as mcolors 

# fig, axes = plt.subplots(2, 2, figsize=(16,10), sharey=True, dpi=160)
# cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]

# for i, ax in enumerate(axes.flatten()):
#     ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
#     ax_twin = ax.twinx()
#     ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
#     ax.set_ylabel('Word Count', color=cols[i])
#     ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 3500)
#     ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
#     ax.tick_params(axis='y', left=False)
#     ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
#     ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

# fig.tight_layout(w_pad=2)    
# fig.suptitle('Word Count and Importance of Topic Keywords', fontsize=22, y=1.05)    
# plt.show()

In [61]:
# get the 50 articles
fifty_articles = df_char['Text'].values

In [62]:
# convert the 2D list to one big string text so we can apply the sequence methodolgy over the text
from itertools import chain

articles_string = []

for article in fifty_articles:
    corpus = ''
    # article[0] take the first element since its a nested list we need only the first ele
    for string in article[0]:
        corpus += " " + string  
    article = corpus
    articles_string.append(corpus)

In [63]:
# eaxmple of article
len(articles_string)

10

In [64]:
#df_char.drop('index', inplace=True)

In [65]:
len(df_char)

10

In [66]:
# save each clean text to our dataframe
df_char['corpus'] = articles_string

/tmp/ipykernel_87712/3978813885.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_char['corpus'] = articles_string


In [67]:
df_char.head()

,Topic_num,Keywords,Text,corpus
index,,,,
0.0,0.0,"[measure, class, predict, true, set, base, pai...","[[evaluation, measure, hierarchical, view, nov...",evaluation measure hierarchical view novel ec...
1.0,1.0,"[domain, learn, modality, target, representati...","[[find, learn, robust, joint, representation, ...",find learn robust joint representation cyclic...
2.0,2.0,"[naive, probability, class, baye, text, featur...","[[naive, bayes, text, introduction, theory, in...",naive bayes text introduction theory introduc...
3.0,3.0,"[test, model, method, system, word, likelihood...","[[find, structure, genome, symbolic, sequence,...",find structure genome symbolic sequence docto...
4.0,4.0,"[word, model, language, vector, train, learn, ...","[[language, model, engineering, apply, abstrac...",language model engineering apply abstract des...


In [68]:
data = df_char[['Topic_num', 'corpus']].to_numpy()

In [69]:
data[0:1]

array([[0.0,
        ' evaluation measure hierarchical view novel economic abstract hierarchical address problem classify item hierarchy class important issue hierarchical evaluation complicate hierarchical relation class several measure propose hierarchical hierarchy way paper study problem hierarchical analyze abstract key component exist performance measure propose alternative generic view hierarchical evaluation introduce correspond novel measure propose state art empirically test large domain text empirical result illustrate desirable behavior exist approach propose method overcome method range case introduction hierarchical address problem classify item class past year mainstream research place enough emphasis presence relation case hierarchical relation gradually change put hierarchical partly real world edge system service hierarchical scheme organize datum research hierarchical become algorithm ill address large scale problem hierarchically relate class initial result large sc

In [70]:
input_features = []
output = []
total_n_v = 0
topic_features = []

# Data contain array [number of topic, text of the article]
# get the features for all articles so remove the data[0:1] to only data
for index, article  in enumerate(data[0:1]):
    # Get the text of the article
    raw_text = article[1]
    # Number of the topic
    print("Topic Number", article[0])
    # Topic features are the keywords with thier wights and counts
    topic_features = [x for x in out if x[1] == article[0]]
    print("Topic features {} for article number {}".format(topic_features, index+1))
    
    # Get the Chars of the article
    chars = sorted(list(set(raw_text)))
    # make dictionary of each charchter that exist in the article and give it index (Chrachter, index)
    char_to_int = dict((c, i) for i, c in enumerate(chars))
    # the same as before but here we have (index,Char)
    int_to_char = dict((i, c) for i, c in enumerate(chars))

    print("The unique Charchters in the article",chars)
    print("Char to int" , char_to_int)
    print("Int to char", int_to_char)
    
    # change the word to char int representation
    for keyword in topic_features:
        word = keyword[0]
        data_x_keyword = [char_to_int[char] for char in word]
        keyword[0] = data_x_keyword
        
    print("Topic features after editing the string to char int", topic_features)

    
    # Prints the total characters and character vocab size
    n_chars = len(raw_text)
    n_vocab = len(chars)
    
    total_n_v += n_vocab

    print("The number of total characters are", n_chars)
    print("\nThe character vocab size is", n_vocab)
    
    #Prepares dataset where the input is sequence of 100 characters and target is next character.
    seq_length = 100

    dataX = []
    dataY = []

    for i in range(0, n_chars - seq_length, 1):
          seq_in = raw_text[i:i + seq_length]
          seq_out = raw_text[i + seq_length]
          dataX.append([char_to_int[char] for char in seq_in])
          dataY.append(char_to_int[seq_out])

    n_patterns = len(dataX)
    print ("Total Patterns: ", n_patterns)
    
    from keras.utils import np_utils
    # reshapes X to be [samples, time steps, features]
    
    X = np.reshape(dataX, (n_patterns, seq_length, 1))
    print("Example of X", X[0])
    print("The X input", len(X))
    # get all the features of the article words
    input_features.append(X)
    # one hot encodes the output variable
    Y = np_utils.to_categorical(dataY)
    print("The length of the Y output", len(Y))
    print("The Y output", Y[1])
    print("The Y output", Y[0])
    
    output.append(Y)   

Topic Number 0.0
Topic features [['measure', 0, 0.025205942, 421], ['class', 0, 0.019185288, 428], ['predict', 0, 0.013641417, 340], ['true', 0, 0.013528929, 176], ['set', 0, 0.01335979, 1063], ['base', 0, 0.012058615, 803], ['pair', 0, 0.011719716, 328], ['hierarchical', 0, 0.009237546, 154], ['figure', 0, 0.009011456, 590], ['system', 0, 0.008318447, 976]] for article number 1
The unique Charchters in the article [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Char to int {' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
Int to char {0: ' ', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u'

In [71]:
# the features are the first element of nested array
input_features[0].shape

(34105, 100, 1)

In [72]:
# features of the Vocab Words
X1= input_features[0]

In [73]:
# Topic Features, keywords only for first Topc since we are trying over first article 
X2 = []
for topic in topic_features:
    X2.append(topic[0])
    

In [74]:
X2 

[[13, 5, 1, 19, 21, 18, 5],
 [3, 12, 1, 19, 19],
 [16, 18, 5, 4, 9, 3, 20],
 [20, 18, 21, 5],
 [19, 5, 20],
 [2, 1, 19, 5],
 [16, 1, 9, 18],
 [8, 9, 5, 18, 1, 18, 3, 8, 9, 3, 1, 12],
 [6, 9, 7, 21, 18, 5],
 [19, 25, 19, 20, 5, 13]]

In [75]:
embedding_wieghts.shape

(10, 10)

In [76]:
# length of the output
len(output[0])

34105

In [77]:
real_Y = output[0]

In [78]:
embedding_dim =100
max_length =100

In [79]:
n_topics = len(topics)
n_topics

10

In [91]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional,Dropout,Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [80]:
# from keras.layers import Dropout
# from keras.layers import Concatenate

In [92]:
# adding two embedding layers one for the Vocabulary and one for the topics 

embedding_layer_1 = Embedding(n_vocab,
                            embedding_dim,
                            input_length=max_length,
                            trainable=False)

embedding_layer_2 = Embedding(n_topics,
                            10,
                            weights= [embedding_wieghts],
                            input_length = 10,
                            trainable=False)


s1rnn = Sequential()
s1rnn.add(embedding_layer_1)
s1rnn.add(LSTM(256,  input_shape=(input_features[0].shape[1], embedding_dim),return_sequences=True))
s1rnn.add(Dropout(0.2))
s1rnn.add(Dense(1))

s2rnn = Sequential()
s2rnn.add(embedding_layer_2)
s2rnn.add(LSTM(256,  input_shape=(10, 10),return_sequences=True))
s2rnn.add(Dropout(0.2))
s2rnn.add(Dense(1))

In [95]:
s1rnn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          2700      
                                                                 
 lstm_2 (LSTM)               (None, 100, 256)          365568    
                                                                 
 dropout_2 (Dropout)         (None, 100, 256)          0         
                                                                 
 dense_6 (Dense)             (None, 100, 1)            257       
                                                                 
Total params: 368,525
Trainable params: 365,825
Non-trainable params: 2,700
_________________________________________________________________


In [96]:
s2rnn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 10, 10)            100       
                                                                 
 lstm_3 (LSTM)               (None, 10, 256)           273408    
                                                                 
 dropout_3 (Dropout)         (None, 10, 256)           0         
                                                                 
 dense_7 (Dense)             (None, 10, 1)             257       
                                                                 
Total params: 273,765
Trainable params: 273,665
Non-trainable params: 100
_________________________________________________________________


In [93]:
model = Sequential()
merged = Concatenate(axis=1)([s1rnn.output,s2rnn.output])
model.add(merged)
model.add(Dense(1))
#model.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=['accuracy'])
model.fit([X1,X2], Y,batch_size=128, nb_epoch=20, validation_split=0.05)

TypeError: The added layer must be an instance of class Layer. Received: layer=KerasTensor(type_spec=TensorSpec(shape=(None, 110, 1), dtype=tf.float32, name=None), name='concatenate_1/concat:0', description="created by layer 'concatenate_1'") of type <class 'keras.engine.keras_tensor.KerasTensor'>.

In [ ]:
X.shape

In [ ]:
model1 = Sequential()
model1.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(Embedding(n_topics, embedding_dim, input_length=max_length))

model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(real_Y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=['accuracy'])
model1.summary()

In [ ]:
model1.fit(input_features, real_Y, epochs = 20, batch_size=128)

In [ ]:
# Generates the sequence similar to above methods. Gets the generated string using the model.
def predict_next_n_chars(pattern, n):
    for i in range(n):
      x = np.reshape(pattern, (1, len(pattern), 1))
      prediction = model1.predict(x, verbose=0)
      print (int_to_char[np.argmax(prediction)], end = '')   #get next char index.
      seq_in = [int_to_char[value] for value in pattern]
      pattern.append(np.argmax(prediction))
      pattern = pattern[1:len(pattern)]

In [ ]:
#picks a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])
print ("Seed -",  input_str, sep = '\n\n')
print ("\nGenerated string -\n")

predict_next_n_chars(pattern, 200)

In [ ]:
input_str = "Many modern service systems rely on a network of hub facilities to help concentrate flows of freight or passengers"

#Uses the first 100 characters from given input_str as input to generate next 200 characters. 
input_str = input_str.lower()
input_string = ''
for each in input_str:
    if each in chars:
           if (len (input_string) < 100):
                input_string += each

pattern = []
pattern.append([char_to_int[char] for char in input_string])

print ("Seed -",  input_str, sep = '\n\n')
print ("\nGenerated string -\n")
predict_next_n_chars(pattern[0], 200)

In [ ]:
input_str = "recently most of the research based on the Neural networks models, because they are fast and accurate models "

#Uses the first 100 characters from given input_str as input to generate next 200 characters. 
input_str = input_str.lower()
input_string = ''
for each in input_str:
    if each in chars:
           if (len (input_string) < 100):
                input_string += each

pattern = []
pattern.append([char_to_int[char] for char in input_string])

print ("Seed -",  input_str, sep = '\n\n')
print ("\nGenerated string -\n")
predict_next_n_chars(pattern[0], 200)